# F215089
## 20 January 2023
### Course work processing Program

- This program contains functions to clean the given attendance records and store them into the SQLite tables
- Warning: the read_data function relies on the path of the file being the data folder. 
- The dataframe was also cleaned before storing the column names into a dataframe. this was done so that column names not useful to the dataset will not be stored.
- The last cell calls the function makes use of all the functions and gives the results required by the coursework.

## Importing Packages

In [1]:
import sqlite3
import pandas as pd
import numpy as np

## Dataframe Preparations

In [2]:
def read_data(filename):
    """
    This function reads the csv file from the data folder
    input:
        filename: the name of the scv file
    return:
        df: the pandas dataframe of the file
    """
    file = filename +".csv"
    path = path = "./data/" #Folder Path
    df=pd.read_csv(path+file, index_col=0)
    return df

def clean_values(dfData):
    """This function cleans the dataset
    input:
       dfData: the dataframe to be cleaned 
    return:
        dfClean: a clean dataframe
    """
    
    dfClean = dfData.copy()
    dfClean = dfClean.replace({"GPS": True, "X": False, "Ex": np.nan})
    dfClean = dfClean.astype("boolean") #apply the specific datatype
    dfClean.dropna(axis = 1, how = "all", inplace = True)
    dfClean.dropna(axis = 0, how = "all", inplace = True)  
    return dfClean 


def colnames_df(dfData):
    """
    This function saves the column names of a dataframe
    to a dataframe 
    input:
        dfData: the dataframe
    return:
        df_session : returns the column names as a dataframe
    """
    df_session = dfData.copy()
    df_session.columns = df_session.columns.str.strip()
    df_session = df_session.columns
    df_session = df_session.map(lambda x: x.split("\n"))
    df_session = list(df_session)
    df_session = pd.DataFrame(df_session)
    df_session.columns = ["session", "day_date", "time", "lecture_type", "venue"]
    df_week = df_session["session"].str.split(".", expand = True) 
    df_session["week"] = df_week[1]
    df_session["session"] = df_week[0]
    df_session = df_session[["session","week", "day_date", "time", "lecture_type", "venue"]]
    df_session.index.names = ["sess_id"]
    return df_session

def rename_columns(df):
    """ 
    changes the column names to unique numeric session ids to the dataframe
    
    input:
        df: dataframe
    return:
        df_new_col_names: dataframe with unique id sessions 
        and 
        df_new_col_namesT: transposed dataframe with unique sessions id
    """
    df_new_col_names = df.copy()
    df_new_col_names.columns=np.arange(len(df_new_col_names.columns))
    df_new_col_namesT= df_new_col_names.transpose()
    df_new_col_namesT.index.names=["sess_id"]#index becomes sess_id i.e session id
    return df_new_col_names, df_new_col_namesT

## Database Operations

In [3]:
def save_to_db(dataframe, tablename):
    '''
    saves the dataframe to the Database
    input:
        dataframe
        tablename: the name you want to store the dataframe as in Database
    '''
    df = dataframe
    conn = sqlite3.connect('CWDatabase.db')
    df.to_sql(tablename,if_exists='replace',
                  index=True,
                  con=conn)
    
    conn.close()

## Main code

In [4]:
def main_cw_preprocessing():
    #reading the COA111 module dataset
    
    dfCOA111 = read_data("22COA111ModuleRegister")
    dfCleanCOA111 = clean_values(dfCOA111)
    dfCleanCOA111N, dfCleanCOA111T = rename_columns(dfCleanCOA111)
    dfCOA111Sessions = colnames_df(dfCleanCOA111)
    
    #reading the COA122 module dataset
    dfCOA122 = read_data("22COA122ModuleRegister")
    dfCleanCOA122 = clean_values(dfCOA122)
    dfCleanCOA122N, dfCleanCOA122T = rename_columns(dfCleanCOA122)
    dfCOA122Sessions = colnames_df(dfCleanCOA122)
    
    #Saving to database
    
    save_to_db(dfCOA111Sessions, "COA111Sessions")
    save_to_db(dfCleanCOA111T, "COA111T")#transposed dataframe
    save_to_db(dfCleanCOA111N, "COA111N")#untransposed dataframe
    
    save_to_db(dfCOA122Sessions, "COA122Sessions")
    save_to_db(dfCleanCOA122T, "COA122T")#transposed dataframe
    save_to_db(dfCleanCOA122N, "COA122N")#untransposed dataframe
 #calling main function   
main_cw_preprocessing()
      